In [ ]:
import matplotlib.pyplot as plt
import math 

################################################################################
# IntroToCSE
# IVPlib_rev3
#

"""
This Python library is useful in solving Initial Value Problems (IVP).

It implements an IVP base class which defines an IVP in the form:
    
    du/dt = f(u,t,p)   for u(tI) = uI

In which the problem would be solved from t=tI to tF and p are a set of 
parameters

Notes: This rev of IVPlib includes:
    * FEsolve: Forward Euler implementation to solve an IVP
    * virtual evalf (to be defined in a subclass)
    * getters
    * __len__
"""

import copy

class IVP():
    def __init__(self, uI, tI, tF, p):
        """
        Args:
            uI (float list): initial condition of state.
            tI (float): initial time.
            tF (float): final time.
            p (dictionary): set of fixed parameters.
        """
        
        self._uI = uI[:]
        self._tI = tI
        self._tF = tF
        self._p  = copy.deepcopy(p)

############ (other) dunder methods ############

    def __len__(self):
        """
        len is defined as number of states (in _uI)
        """
        return len(self._uI)
        
############ virtual methods for use outside of class ############

    def evalf(self,u,t):
        """
        Args:
            u (float list): current solution. 
            t (float): current time.

        Returns:
            float list: f(u,t).
        """
        raise NotImplementedError("evalf is not implemented for this object")

############ getter methods ############

    def get_tI(self):
        """
        Returns:
            float: initial time.
        """
        return self._tI

    def get_tF(self):
        """
        Returns:
            float: final time.
        """
        return self._tF

    def get_uI(self):
        """
        Returns:
            float list: initial state
        """
        return self._uI[:]

    def get_p(self, name):
        """
        Arg:
            name (key): a key which should be in the object's parameter
            dictionary

        Returns:
            value of parameter key given by name
        """
        return self._p[name]

################################################################################
## Functions to numerically integrate an IVP
################################################################################

def FEsolve(thisIVP, dt):
    """
    Solves an IVP using the Forward Euler method with timestep dt. Integrate
    from t=tI until v(tn) is determined for which tn >= tF.

    Args:
        thisIVP (IVP object): object describing IVP being solved.
        dt (float): time increment.

    Returns:
        t (float list): time values at which u(t) is approximated. The nth item in
            the list is the time of the nth step, tn = t[n].
        v (list of float lists): The values of the states at each step.  The nth
            item in the list is the values of the states at tn.  i.e. v(tn) = v[n]
            where v[n] is a float list.  So, if there are three equations being integrated, then
            v[n][0], v[n][1], and v[n][2] are the values of the three states at time t=t[n]

    IMPORTANT: The first element in the returned t and v lists will be the initial values
    of t and v.  Thus:
        * t[0] will be a float which is equal to thisIVP.get_tI()
        * v[0] will be a float list which is equal to thisIVP.get_uI()
    """

    # Sets initial condition & places them in an array
    tI = thisIVP.get_tI()
    t = [tI]

    vI = thisIVP.get_uI()
    v = [vI]

    # Loop from t=tI to t>=tF
    tn = tI
    vn = vI
    tF= thisIVP.get_tF()
    while (tn<tF):
        fn = thisIVP.evalf(vn,tn)
        vn1 = []
        for i in range(len(vn)):
            vn1.append(vn[i] + dt*fn[i])

        v.append(vn1)
        vn = vn1
        
        tn += dt
        t.append(tn)

    return t, v

def RK2_MEsolve(thisIVP, dt):
    """
    Solves an IVP using the RK2 Modified Euler method with timestep dt. Integrate
    from t=tI until v(tn) is determined for which tn >= tF.

    Args:
        thisIVP (IVP object): object describing IVP being solved.
        dt (float): time increment.

    Returns:
        t (float list): time values at which u(t) is approximated. The nth item in
            the list is the time of the nth step, tn = t[n].
        v (list of float lists): The values of the states at each step.  The nth
            item in the list is the values of the states at tn.  i.e. v(tn) = v[n]
            where v[n] is a float list.  So, if there are three equations being integrated, then
            v[n][0], v[n][1], and v[n][2] are the values of the three states at time t=t[n]

    IMPORTANT: The first element in the returned t and v lists will be the initial values
    of t and v.  Thus:
        * t[0] will be a float which is equal to thisIVP.get_tI()
        * v[0] will be a float list which is equal to thisIVP.get_uI()
    """
    
    # Sets initial condition & places them in an array
    tI = thisIVP.get_tI()
    t = [tI]

    vI = thisIVP.get_uI()
    v = [vI]

    # Loop from t=tI to t>=tF
    tn = tI
    vn = vI
    tF= thisIVP.get_tF()
    while (tn<tF):
        va = vn
        ta = tn
        fa = thisIVP.evalf(va,ta)
        a = []
        for i in range(len(vn)):
            a.append(dt*fa[i])
            
        vb = []
        for i in range(len(vn)):
            vb.append(vn[i] + 0.5*a[i])
        tb = tn + 0.5*dt
        fb = thisIVP.evalf(vb,tb)
        b = []
        for i in range(len(vn)):
            b.append(dt*fb[i])
            
        vn1 = []
        for i in range(len(vn)):
            vn1.append(vn[i] + b[i])
            
        v.append(vn1)
        vn = vn1
        
        tn += dt
        t.append(tn)

    return t, v

################################################################################
# Model cooling of coffee in a cup with a simple lumped model
#
#    mc cc dTc/dt = h A (Tout - Tc)
#
# where: mc = mass of coffee in cup
#        cc = heat capacity of coffee
#        Tc = temperature of coffee
#      Tout = temperature of environment
#         h = heat transfer coefficient
#         A = surface area of coffee cup (including upper surface)
#
#  Note that dividing through by mc cc gives the equation implemented:
#
#         dTc/dt = h A / (mc cc) (Tout - Tc)
#

class coffeeIVP(IVP):
    def evalf(self, Tc, t):
        """        
        Args:
            Tc (float list): current temperature of coffee.
            t (float): current time
    
        Returns:
            f (float list): returns dTc/dt
        """

        mc = self.get_p('mc')
        cc = self.get_p('cc')
        h = self.get_p('h')
        A = self.get_p('A')
        Tout = self.get_p('Tout')

        f = h*A/(mc*cc)*(Tout - Tc[0])

        return [f]


################################################################################
# Model cooling of coffee in a cup with a simple lumped model
#
#    mc cc dTc/dt = h A (Tout - Tc)
#
# where: mc = mass of coffee in cup
#        cc = heat capacity of coffee
#        Tc = temperature of coffee
#      Tout = temperature of environment
#         h = heat transfer coefficient
#         A = surface area of coffee cup (including upper surface)
#
#  Note that dividing through by mc cc gives the equation implemented:
#
#         dTc/dt = h A / (mc cc) (Tout - Tc)
#

mc   = 0.35 # kg
cc   = 4200.0 # J / (kg C)
h    = 5.0 # W/(m^2 C)
A    = 0.04 # m^2
Tout = 25.0 # C
        
TcI   = 85.0 # Initial temperature of coffee (C)
tFmin = 700.0 # final time to simulate to (min)
dtmin = 5e1 # time increment to give solutions at (min)

# Convert times to seconds
tF = tFmin*60
dt = dtmin*60

# Initialize CoffeeIVP object
p = {}
p['h']    = h
p['A']    = A
p['mc']   = mc
p['cc']   = cc
p['Tout'] = Tout

coffeeIVP_hotday = coffeeIVP([TcI], 0.0, tF, p)

# Solve coffee IVP
t, v = FEsolve(coffeeIVP_hotday, dt)

# Calculate exact solution
# and extract Tc (list of floats) from v (list of lists)
u  = []
Tc = []

lam  = -h*A/(mc*cc) # Needed for exact solution

for n in range(len(t)):
    ts = t[n]
    t[n] = t[n]/60.0 # convert to minutes
    un = Tout + (TcI-Tout)*math.exp(lam*ts) # this is the exact solution
    u.append(un)
    Tcn = v[n][0]
    Tc.append(Tcn)


# Plot   
# fig, ax = plt.subplots()
# ax.scatter(t,Tc,marker='o',label='numerical')
# ax.set_xlabel('t (min)')
# ax.set_ylabel('$T_c$ (C)')
# ax.grid(True)
    
# ax.scatter(t,u,marker='x',label='exact')
# ax.legend()
plt.figure()

# the arguments to subplot are
# plt.subplot("number of rows", "number of columns", "subplot index")

# this is the "first" subplot. all the plotting commands below are restricted to this scope.
plt.subplot(2,1,1)
plt.scatter(t,Tc,marker='o',label='numerical')

# we enclose in $$ to use LaTeX, don't worry about this
plt.ylabel('$T_c$ (C)')
plt.grid(True)

plt.scatter(t,u,marker='x',label='exact')
plt.title('$\Delta t$ = 50 min')
plt.legend()

# this is the "second" subplot. all the plotting commands below are restricted to this scope.
plt.subplot(2,1,2)
plt.scatter(t,e,marker='o')
plt.xlabel('t (min)')
plt.ylabel('error (C)')
plt.grid(True) 

In [1]:
h=5
A=0.04
mc=0.35
cc=4200
Tout=25
Tc=40

f = h*A/(mc*cc)*(Tout - Tc)
print(f)
print(f"{f:.2E}")

-0.002040816326530612
-2.04E-03


In [ ]:
p = 9869856524432386778.864956560870
print(p)
print(f"{p:.2E}")

In [ ]:
h=5
A=0.04
mc=0.35
cc=4200
Tout=5
Tc=40

f = h*A/(mc*cc)*(Tout - Tc)

print(f)
print(f"{f:.2E}")

In [ ]:
a=2
b=0.01
c=0.1
m=1
r=500
f=350

rabbit= (a*r) - (b*r*f)

fox = (-m*f) + (c*b*r*f)


#print(f"{rabbit}")
print(f"{rabbit:.2E}")


#print(f"{fox}")
print(f"{fox:.2E}")

In [2]:
u = [1, 2, 3]
v = [4, 5, 6]
w = [u[0]+v[0], u[1]+v[1], u[2]+v[2]] # i.e. w = [5,7,9]

print( u+v == w )

False
